#### DNN 기반 이진분류 모델 구현
- 사용되는 데이터셋 : iris.csv
- feature : 3개
- target : 1개 Setosa와 나머지
- 학습방법 : 지도학습 -> 분류 > 이진분류
- 알고리즘 : 인공신경망(ANN) -> 심층(은닉층) 신경망 -> MLP(층이여러개), DNN(은닉층이 많은 구성) 
- FramWork : Pytorch

In [22]:
# 모듈 로딩
# 모델관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score
from torchmetrics.classification import BinaryConfusionMatrix
from torchinfo import summary

# 데이터 전처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [23]:
# 활용 패키지 버전 체크
def versioncheck():
    print(f' torch  {torch.__version__}')
    print(f' pandas  {pd.__version__}')

versioncheck()

 torch  2.4.1
 pandas  2.0.3


In [24]:
DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE)
irisdf

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [25]:
### 타겟 변경 => 정수화, 클래스 3개 => 2개
irisdf['variety'].unique()
irisdf['variety'] == 'Setosa'

0       True
1       True
2       True
3       True
4       True
       ...  
145    False
146    False
147    False
148    False
149    False
Name: variety, Length: 150, dtype: bool

In [26]:
irisdf['variety']=(irisdf['variety'] == 'Setosa')
irisdf['variety']=irisdf['variety'].astype('int')
irisdf['variety']
irisdf

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0
146,6.3,2.5,5.0,1.9,0
147,6.5,3.0,5.2,2.0,0
148,6.2,3.4,5.4,2.3,0


[2] - 모델 클레스 설계 및 정의 - <hr>
어떤 클래스를 만들까? 고려
클래스 목적 : iris.DataSet 학습 후 추론
클래스 이름 : IrisBCFModel
부모 클래스 : nn.Module
매개 변수   : 층별 입출력 갯수 고정하기 때문에 필요 없음
클래스 속성 : featureDF, targetDF, n_rows, n_features
클래스 기능 : __init__() : 모델 구조, forward() : 순방향 학습 <= 오버라이딩(overriding) 상속관계일 때

클래스 구조  
    -입력층 : 피쳐 4개  퍼셉트론 : 50개(보통 입력 때 많이 주고 갈수록 줄임) (4,10)
    -은닉층 : 입력 10개     출력 5개   (10,5)
    -출력층 : 입력5개      타겟(이진분류) 1개    (5,1)  
  
-손실 함수/ 활성화 함수
    *클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드에 사용
    *함수 형태 ==> torch.nn.functional => forward()메서드에 사용

In [27]:
class IrisBFCModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4, 10)
        self.hidden_layer=nn.Linear(10, 5)
        self.out_layer=nn.Linear(5, 1)
    
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y=self.in_layer(input_data) # f1w1+f2w2+f3w3+b 요런 식이 10개(숫자10개)
        y=F.relu(y)                   # 범위 0이상
        
        # 은닉층 : 10개의 숫자 받아오기
        y=self.hidden_layer(y)
        y=F.relu(y)

        # 출력층 : 5개의 숫자 값 => sigmoid 이진분류
        return F.sigmoid(self.out_layer(y))

In [28]:
# 모델 인스턴스 생성
model = IrisBFCModel()
print(model)


IrisBFCModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [29]:
summary(model)

Layer (type:depth-idx)                   Param #
IrisBFCModel                             --
├─Linear: 1-1                            50
├─Linear: 1-2                            55
├─Linear: 1-3                            6
Total params: 111
Trainable params: 111
Non-trainable params: 0

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataSet
- 부모클래스 : utils.data.DataSet
- 속성__필드 : featureDF, targetDF, n_rows, n_featrues  
- 필수 메서드:   
    *__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정  
    *__len__(self) : 데이터의 개수 반환  
    *__getItem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [30]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [31]:
# [테스트] 데이터셋 인스턴스 생성
featureDF = irisdf[irisdf.columns[:-1]] # 2D (150,3)
targetDF = irisdf[irisdf.columns[-1:]] # 1D(150,1)


irisDS=IrisDataset(featureDF,targetDF)

# 데이터로더 인스턴스 생성
irisDL = DataLoader(irisDS, batch_size=1)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비
- 학습 횟수 : EPOCH ( 처음부터 끝까지 공부할 횟수 )
- 배치 크기 : BATCH_SIZE(학습량)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 GPU/CPU)
- 학 습 률  : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1

In [32]:
# 학습 진행 관련 설정 값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스 : 모델, 데이터 셋, 최적화 (, 성능지표) 

In [33]:
# 모델 인스턴스
# 학습용, 검증용, 테스트용 데이터 분리
model = IrisBFCModel()

# 학습용, 검증용, 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{y_train.value_counts()/y_train.shape[0]} {y_test.value_counts()/y_test.shape[0]} {y_val.value_counts()/y_val.shape[0]}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

# 학습용, 검증용, 테스트용 데이터셋 생성
trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 학습용 데이터 로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

# 최적화

(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)
variety
0          0.702381
1          0.297619
Name: count, dtype: float64 variety
0          0.657895
1          0.342105
Name: count, dtype: float64 variety
0          0.571429
1          0.428571
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


- 최적화 & 손실함수 인스턴스 생성하기

In [34]:

# 최적화 인스턴스 => w, b model.parameter 전달
optimizer=optim.Adam(model.parameters(), lr=LR)


# 바이너리용 손실함수 인스턴스 => 분류
# 예측값을 확률값으로 전달함 ==> sigmoid() AF 처리 후 전달
reqLoss=nn.BCELoss()

[5] 학습진행

In [35]:
len(trainDL), trainDL.__len__()
## 모델 파일 관련

### models 폴더 아래 프로젝트 폴더 아래 모델 파일을 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/BCF/'

# 저장 파일명
SAVE_FILE = SAVE_PATH + 'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL=SAVE_PATH+'model_all.pth'

# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더 / 폴더 / ... 하위 폴더까지 생성

In [37]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT = irisDS.n_rows / BATCH_SIZE
print(f'CNT => {CNT}')

# 학습 모니터링 / 스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT = 0
LIMIT_VALUE = 9
# 학습 모드로 모델 설정
model.train()

for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0

    for featureTS, targetTS in trainDL:

        #학습 진행
        pre_y=model(featureTS)

        #손실 계산
        loss=reqLoss(pre_y, targetTS)
        loss_total += loss.item()


        #성능평가 계산
        score=F1Score(task='binary')(pre_y, targetTS)
        score_total += score.item()

        #최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에폭당 검증 기능을 키겠다.
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        #평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqLoss(pre_val, val_targetTS)

        # 성능평가
        score_val = F1Score(task='binary')(pre_val, val_targetTS)

    # 에폭당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

        # # 손실 기준
    if len(LOSS_HISTORY[0]) >= 2:
       if LOSS_HISTORY[1][-1] <= LOSS_HISTORY[1][-2] : BREAK_CNT +=1

    # 성능 기준
    if len(SCORE_HISTORY[1]) == 1: # 첫번째 횟수 저장
       torch.save(model.state_dict(), SAVE_FILE)

       # 모델 전체 저장
       torch.save(model, SAVE_MODEL)
    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]) : # 첫번째 점수랑 두번째 점수 비교 후 더 성능이 큰쪽을 저장
            torch.save(model.state_dict(), SAVE_FILE)
            torch.save(model, SAVE_MODEL)

    # 성능이 좋은 학습 가중치 저장

    # 학습중단
    if BREAK_CNT >10:
        print('성능 및 손실 개선이 없어서 학습 중단') 
        break

    

CNT => 15.0
성능 및 손실 개선이 없어서 학습 중단


In [83]:
print(LOSS_HISTORY)
print(SCORE_HISTORY)

[[0.5720636069774627, 0.5653235971927643, 0.5592088580131531, 0.5531753659248352, 0.5468607395887375, 0.5392442911863327, 0.5270370841026306, 0.514256164431572, 0.5033300220966339, 0.49443804919719697, 0.48674654960632324, 0.47911277413368225, 0.47130192816257477, 0.4632317006587982, 0.45495554208755495, 0.44640432894229887, 0.4375766098499298, 0.4287372827529907, 0.42016775012016294, 0.41176167130470276, 0.4032977223396301, 0.3949081748723984, 0.386570480465889, 0.37834618985652924, 0.3702303320169449, 0.36220566034317014, 0.35438972562551496, 0.3467731952667236, 0.3390445336699486, 0.33218401968479155, 0.3258331149816513, 0.3199864149093628, 0.3143597334623337, 0.3089700400829315, 0.30375444889068604, 0.298791241645813, 0.2941936954855919, 0.2897902399301529, 0.28556986153125763, 0.28151207566261294, 0.2776397839188576, 0.27402557283639906, 0.27054073214530944, 0.2671940788626671, 0.2639585554599762, 0.2608431622385979, 0.25791520476341245, 0.25505183860659597, 0.25227373018860816, 0

[6] 학습결과 시각화
- 학습과 검증의 Loss 변화, 성능 변화 체크